In [ ]:
import pandas as pd
import datetime

# Requête SQL utilisée pour obtenir le tsv

```SQL
SELECT ticket.id AS ticket_id, 
  queue.name AS queue, 
  ticket_state_type.name AS state_type, 
  ticket.create_time AS ticket_create_time, 
  ticket.change_time AS ticket_change_time, 
  article_type.name AS article_type,
  article_sender_type.name AS article_sender_type,
  article.create_time AS article_create_time
FROM ticket 
  INNER JOIN queue on queue.id=ticket.queue_id 
  INNER JOIN ticket_state ON ticket.ticket_state_id=ticket_state.id 
  INNER JOIN ticket_state_type ON ticket_state.type_id=ticket_state_type.id 
  INNER JOIN article ON article.ticket_id=ticket.id 
  INNER JOIN article_type ON article.article_type_id=article_type.id
  INNER JOIN article_sender_type ON article.article_sender_type_id=article_sender_type.id
WHERE 
  ticket_state_type.name='closed'
ORDER BY article_create_time;
```


In [ ]:
df_tickets_data = pd.read_csv('tickets_data.tsv',sep='\t')
for c in ["ticket_create_time", "article_create_time", "ticket_change_time"]:
    df_tickets_data[c] = pd.to_datetime(df_tickets_data[c])
df_tickets_data.head()

In [ ]:
(df_tickets_data["ticket_change_time"] - df_tickets_data["ticket_create_time"]).median()

In [ ]:
group_by_tickets = df_tickets_data.groupby("ticket_id")


In [ ]:
df_first_customer_item = df_tickets_data[df_tickets_data.article_sender_type == 'customer'].groupby("ticket_id")["article_create_time"].min().to_frame("first_customer_item_time").reset_index()
df_first_agent_email = df_tickets_data[(df_tickets_data.article_sender_type == 'agent') & (df_tickets_data.article_type == 'email-external')].groupby("ticket_id")["article_create_time"].min().to_frame("first_agent_email_time").reset_index()
df_last_agent_email = df_tickets_data[(df_tickets_data.article_sender_type == 'agent') & (df_tickets_data.article_type == 'email-external')].groupby("ticket_id")["article_create_time"].max().to_frame("last_agent_email_time").reset_index()

In [ ]:
df_first_customer_item.reset_index().head()

In [ ]:
df_tickets_data_processed = df_tickets_data
df_tickets_data_processed = pd.merge(left=df_tickets_data_processed, right=df_first_customer_item, left_on="ticket_id", right_on="ticket_id")
df_tickets_data_processed = pd.merge(left=df_tickets_data_processed, right=df_first_agent_email, left_on="ticket_id", right_on="ticket_id")
df_tickets_data_processed = pd.merge(left=df_tickets_data_processed, right=df_last_agent_email, left_on="ticket_id", right_on="ticket_id")
df_tickets_data_processed.head()

In [ ]:
df_tickets_data_processed["first_response_hours"] = (df_tickets_data_processed["first_agent_email_time"] - df_tickets_data_processed["first_customer_item_time"]) / pd.Timedelta('1 hour')
df_tickets_data_processed["solution_hours"] = (df_tickets_data_processed["last_agent_email_time"] - df_tickets_data_processed["first_customer_item_time"]) / pd.Timedelta('1 hour')
df_tickets_data_processed["close_hours"] = (df_tickets_data_processed["ticket_change_time"] - df_tickets_data_processed["first_customer_item_time"]) / pd.Timedelta('1 hour')


In [ ]:
valid_tickets_data = df_tickets_data_processed[(df_tickets_data_processed.first_response_hours > 0) & (df_tickets_data_processed.solution_hours > 0)]
no_duplicate_data = valid_tickets_data.drop_duplicates(subset=["ticket_id"])

In [ ]:
print("First response time (hours):" + str(no_duplicate_data["first_response_hours"].median()))
print("Solution time (hours):" + str(no_duplicate_data["solution_hours"].median()))
print("Close time (hours):" + str(no_duplicate_data["close_hours"].median()))

In [ ]:
no_duplicate_data.groupby("queue")["first_response_hours"].median()

In [ ]:
no_duplicate_data["year"] = no_duplicate_data["ticket_create_time"].map(lambda x: x.year)

In [ ]:
def simplified_queue(x):
    if "Colosse" in x or "UL" in x: return "Colosse" 
    if "Helios" in x: return "Helios"
    if "Guillimin" in x: return "Guillimin"
    if "Altix" in x: return "Altix" 
    if "Cottos" in x: return "Cottos"
    if "Hades" in x: return "Hades"
    if "Briaree" in x: return "Briaree"
    if "UdeM" in x: return "Briaree"
    if "UdeS" in x: return "Mammouth"
    if "Communications" in x: return "Communications"
    if "Formations" in x: return "Formations"
    if "Hebergement" in x: return "Hebergement"
    if "Demandes FRQ" in x: return "Demandes FRQ"
    if "Support" in x or "clumeq" in x or "CalculQuebec" in x: return "General"
    if "Raw" in x or "Staff" in x: return "Other"
    return x

no_duplicate_data["simplified_queue"] = no_duplicate_data["queue"].map(simplified_queue)

In [ ]:
no_duplicate_data.groupby(["year","simplified_queue"])["first_response_hours"].mean().to_frame("mean_first_response_time").reset_index()
no_duplicate_data.groupby(["year","simplified_queue"])["first_response_hours"].median().to_frame("median_first_response_time").reset_index()


In [ ]:
s1 = pd.concat([
    no_duplicate_data.groupby(["year","simplified_queue"])["first_response_hours"].mean().to_frame("mean_first_response_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["first_response_hours"].median().to_frame("median_first_response_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["solution_hours"].mean().to_frame("mean_solution_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["solution_hours"].median().to_frame("median_solution_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["close_hours"].mean().to_frame("mean_close_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["close_hours"].median().to_frame("median_close_hours"),
    no_duplicate_data.groupby(["year","simplified_queue"])["close_hours"].count().to_frame("count"),

    ],
    axis=1
).reset_index()
s2 = pd.concat([
    no_duplicate_data.groupby(["year"])["first_response_hours"].mean().to_frame("mean_first_response_hours"),
    no_duplicate_data.groupby(["year"])["first_response_hours"].median().to_frame("median_first_response_hours"),
    no_duplicate_data.groupby(["year"])["solution_hours"].mean().to_frame("mean_solution_hours"),
    no_duplicate_data.groupby(["year"])["solution_hours"].median().to_frame("median_solution_hours"),
    no_duplicate_data.groupby(["year"])["close_hours"].mean().to_frame("mean_close_hours"),
    no_duplicate_data.groupby(["year"])["close_hours"].median().to_frame("median_close_hours"),
    no_duplicate_data.groupby(["year"])["close_hours"].count().to_frame("count"),
    ],
    axis=1
).reset_index()
s2["simplified_queue"] = "TOTAL"

result = pd.concat([s1,s2],axis=0)
result.to_csv("tickets_donnees_analysees.csv")

In [ ]:
mean_first_response = {}
median_first_response = {}
mean_solution = {}
median_solution = {}
mean_close = {}
median_close = {}

for queue in ["Guillimin","Briaree","Colosse","Helios"]:
    for year in [2014,2015,2016,2017]:
        df_year = no_duplicate_data[(no_duplicate_data.ticket_create_time > str(year) + '-01-01') & (no_duplicate_data.ticket_create_time < str(year+1) + '-01-01')]
        df_queue_year = df_year[df_year.queue.str.contains(queue)]
        df = df_queue_year
        mean_first_response[year] = df["first_response_hours"].mean()
        median_first_response[year] = df["first_response_hours"].median()
        mean_solution[year] = df["solution_hours"].mean()
        median_solution[year] = df["solution_hours"].median()
        mean_close[year] = df["close_hours"].mean()
        median_close[year] = df["close_hours"].median()
    print("Queue:" + queue)
    print("Moyenne première réponse:" + str(mean_first_response))
    print("Médiane première réponse:" + str(median_first_response))
    print("Moyenne solution:" + str(mean_solution))
    print("Médiane solution:" + str(median_solution))
    print("Moyenne fermeture:" + str(mean_close))
    print("Médiane fermeture:" + str(median_close))
    print("--")

    
for year in [2014,2015,2016,2017]:
    df_year = no_duplicate_data[(no_duplicate_data.ticket_create_time > str(year) + '-01-01') & (no_duplicate_data.ticket_create_time < str(year+1) + '-01-01')]
    df = df_year
    mean_first_response[year] = df["first_response_hours"].mean()
    median_first_response[year] = df["first_response_hours"].median()
    mean_solution[year] = df["solution_hours"].mean()
    median_solution[year] = df["solution_hours"].median()
    mean_close[year] = df["close_hours"].mean()
    median_close[year] = df["close_hours"].median()
    

In [ ]:
print("Moyenne première réponse:" + str(mean_first_response))
print("Médiane première réponse:" + str(median_first_response))
print("Moyenne solution:" + str(mean_solution))
print("Médiane solution:" + str(median_solution))
print("Moyenne fermeture:" + str(mean_close))
print("Médiane fermeture:" + str(median_close))